In [ ]:
!pip install sparqlwrapper

Get prompt.

In [14]:
#user_prompt = input("Enter a prompt: ")
user_prompt = "Select all cities of Greece and their population"

prompt = f"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph. The query should be enclosed by three backticks on new lines, denoting that it is a code block.
Human: {user_prompt}
Generator: ```"""

Pass the prompt to the LLM.

In [3]:
import torch

def run_inference(model, tokenizer, prompt):
    results = []
    
    if tokenizer == None:
        # Generate output
        with torch.no_grad():
            outputs = model(prompt)
            
        # Decode and print output
        print("Prompt:", prompt)
        print("Generated text:" + outputs + "\n")
        results.append("Generated text:" + outputs)
    else:
        # Move model to GPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()  # Set model to evaluation mode
            
        # Tokenize prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
            
        # Generate output
        with torch.no_grad():
            outputs = model.generate(**inputs, 
                            max_length=500,  # Set a maximum length for generated text
                            #do_sample=True,  # Enable sampling
                            #top_k=7,        # Top-k sampling
                            #top_p=0.1,      # Top-p sampling (nucleus sampling)
                            #num_return_sequences=1,
                            #repetition_penalty=1, # No penalty for instruction tuned models.
                            repetition_penalty=1.2, # Penalty on repeating tokens.
                            eos_token_id=tokenizer.eos_token_id,  # Specify EOS token ID
                            pad_token_id=tokenizer.pad_token_id  # Specify PAD token ID
                            )
        
        # Extract generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove the prompt text
        prompt_length = len(prompt)
        generated_text = generated_text[prompt_length:]

        # Decode and print output
        print("Prompt:", prompt)
        print(generated_text)
        results.append(generated_text)
    
    # Clear model from RAM
    del model
    torch.cuda.empty_cache()
    
    return results

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("alpindale/Mistral-7B-v0.2-hf", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("alpindale/Mistral-7B-v0.2-hf")

In [ ]:
results = run_inference(model, tokenizer, prompt)

end_index = results[0].find("```")

# Extract the substring from the start of the string up to the first occurrence of ```
if end_index != -1:
    query = results[0][:end_index]
else:
    # If ``` is not found, keep the original string
    query = results[0]

# Now remove the SPARQL prefix that the model adds.
start_index = query.find("SPARQL")
if start_index == 0:
    # Remove the prefix and all characters leading up to it
    query = query[start_index + len("SPARQL"):]

print("QUERY: ", query)

Run the generated SPARQL query against a Dbpedia endpoint and get the results.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Set the DBpedia endpoint URL
endpoint_url = "http://dbpedia.org/sparql"

# Create a SPARQLWrapper object, specifying the endpoint URL
sparql = SPARQLWrapper(endpoint_url)

# Define your SPARQL query
# sparql_query = """
#     SELECT ?country ?population
#     WHERE {
#         ?country rdf:type dbo:Country ;
#                  dbo:populationTotal ?population .
#         FILTER (?population > 5000000000)
#     }
#     LIMIT 10
# """

sparql_query = query

# Set the SPARQL query string
sparql.setQuery(sparql_query)

# Set the query type (in this case, it's a SELECT query)
sparql.setReturnFormat(JSON)

# Execute the SPARQL query and parse the results
try:
    # Execute the query and convert the result into JSON format
    results = sparql.query().convert()
    
    print(results)
    # Process the results
#     for result in results["results"]["bindings"]:
#         country_name = result["country"]["value"]
#         population = result["population"]["value"]
#         print(f"Country: {country_name}, Population: {population}")

except Exception as e:
    print(f"Error executing SPARQL query: {e}")